# ProxyWhirl: Getting Started

A short, runnable walkthrough that sets up a proxy pool, rotates through it, and performs a mocked request without requiring live proxies.

## Installation
From the repository root (recommended):

```bash
uv sync --group dev --extra storage
```

Or from PyPI:

```bash
uv pip install "proxywhirl[storage]"
```

In [ ]:
import proxywhirl

print("ProxyWhirl version:", proxywhirl.__version__)

## Build a Proxy Pool
Define a few sample proxies. Credentials stay encrypted internally via `SecretStr`.

In [ ]:
from pydantic import SecretStr

from proxywhirl import Proxy, ProxyPool, ProxyRotator

pool = ProxyPool(
    name="getting-started",
    proxies=[
        Proxy(url="http://proxy1.local:8080", country_code="US"),
        Proxy(url="http://proxy2.local:8080", country_code="DE"),
    ],
)

pool.add_proxy(
    Proxy(
        url="http://auth-proxy.local:9000",
        username=SecretStr("user"),
        password=SecretStr("pass"),
        metadata={"tier": "premium"},
    )
)

rotator = ProxyRotator(proxies=[p.model_copy(deep=True) for p in pool.get_all_proxies()])
print(f"Pool size: {rotator.pool.size}")

## Simulate a Proxied Request
Patch `httpx.Client` with `MockTransport` to keep everything offline while still exercising the rotation logic.

In [ ]:
import httpx
from unittest.mock import patch

# Track which proxy the strategy selected
picks: list[str] = []
original_select = rotator.strategy.select

def select_with_tracking(pool, context=None):
    proxy = original_select(pool, context=context)
    picks.append(proxy.url)
    return proxy

rotator.strategy.select = select_with_tracking  # type: ignore[attr-defined]

def handler(request: httpx.Request) -> httpx.Response:
    current_proxy = picks[-1] if picks else "unknown"
    return httpx.Response(
        200,
        json={"requested_url": str(request.url), "proxy_used": current_proxy},
        request=request,
    )

_original_client = httpx.Client

def client_factory(*args, **kwargs):
    kwargs["transport"] = httpx.MockTransport(handler)
    return _original_client(*args, **kwargs)

with patch("httpx.Client", client_factory):
    response = rotator.get("https://httpbin.org/ip").json()

rotator.strategy.select = original_select

print("Rotation order:", picks)
print("Sample response:", response)
print("Pool stats:", rotator.get_pool_stats())

## Next Steps
- Swap the mock transport for a real proxy endpoint.
- Try other strategies with `rotator.set_strategy("least-used")` or `rotator.set_strategy("random")`.
- Explore persistence options in `examples/examples.ipynb`.